# 🔍 Advanced Search Demo - Integrating with AI Search MCP Server
This tutorial demonstrates how to implement advanced AI search capabilities by integrating with AI Search MCP Server. The example presents an implementation of AI-driven search using the AI Search MCP Server. It demonstrates how to orchestrate a recursive advanced search loop powered by a language model and the MCP search infrastructure.

## 1. Environment Setup

### 1.1 System Requirments

Before starting, ensure your system meets these requirements:
- Python version over 3.10 is installed.
- Ensure the following Python libraries are included: ```appbuilder```, ```argparse```, ```openai```.

### 1.2 Install Dependencies

In [ ]:
pip install appbuilder-sdk argparse openai

## 2. 🔁 Main Search Structure Overview
Before initiating the search process, the LLM capability is invoked to generate a task plan based on the input query, which serves to guide the subsequent search process and supplement the reference information for input query.

The main Search process contains 3 steps in a loop:
- LLM Decision – Whether need more information to answer the user query (further search or not)
- Generates MCP Server Search Parameters
- Call MCP Server and Update Context

When the loop ends, LLM answers the User query refering the Context by searching.

## 3. Core Funcionality Dependencies
### 3.1 Set up AI Search MCP Endpoint
#### 3.1.1 Get an API Key
You’ll need an API key from [QIANFAN](https://cloud.baidu.com/doc/AppBuilder/s/lm68r8e6i) to call the [AI Search MCP server](https://sai.baidu.com/ai/server/%25E7%2599%25BE%25E5%25BA%25A6AI%25E6%2590%259C%25E7%25B4%25A2/%E7%99%BE%E5%BA%A6?id=e014c6ffd555697deabf00d058baf388).

The API key is sensitive and should be kept secret. You can set it as an environment variable or store it in a .env file in your local folder. We will use the python-dotenv package to load the environment variables from the .env file. 

An example of a .env file is : 
    API_KEY=your_api_key_here

In [72]:
API_KEY = "<AppBuilder API Key>"

#### 3.1.2 Initialize the MCP server
Then let's define the service url and initilize an MCP client:

In [73]:
service_url = f"http://appbuilder.baidu.com/v2/ai_search/mcp/sse?api_key=Bearer+{API_KEY}"

In [75]:
import asyncio
import logging

import nest_asyncio
from appbuilder.mcp_server.client import MCPClient

logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("mcp.client.sse").setLevel(logging.WARNING)

async def call_mcp():
    client = MCPClient()
    await client.connect_to_server(service_url=service_url)
    return client

nest_asyncio.apply()

loop = asyncio.get_event_loop()
client = loop.run_until_complete(call_mcp())

[2025-06-26 21:27:41,312.312] client.py [line:81] INFO [main-12862631056286909524] 
Connected to server with tools:['AIsearch']


INFO:appbuilder:
Connected to server with tools:['AIsearch']


[Tool(name='AIsearch', description='\n    执行搜索。\n\n    Args:\n        query (str): 搜索查询关键词或短语，用于指定需要搜索的内容。支持自然语言查询，可以包含多个关键词。Agent应该根据用户意图构建合适的查询语句，确保搜索结果的准确性和相关性。最佳实践：1) 使用简洁明确的关键词组合；2) 对于复杂查询，可以拆分为多个简单查询；3) 避免使用过于宽泛或模糊的词语。\n        model (str, optional): 指定是否使用大语言模型对搜索结果进行总结，以及使用哪个模型进行总结。默认值为空字符串，表示直接返回原始搜索结果。当指定模型名称时（如"ERNIE-3.5-8K"），系统将使用对应的大模型对搜索结果进行智能总结。此参数会显著影响其他参数的行为：1) 当未指定时，temperature、top_p等LLM相关参数将失效；2) 当指定时，搜索结果会经过LLM处理，可能改变原始结果的呈现方式。最佳实践：1) 对于需要深入分析或总结的查询，建议使用ERNIE-3.5-8K或更高级的模型；2) 对于只需要原始搜索结果的查询，保持默认值即可；3) 根据实际需求选择合适的模型版本。\n        instruction (str, optional): 用于控制搜索结果输出风格和格式的指令参数。Agent可以通过此参数指定特定的输出要求，如：结果排序方式、内容过滤条件、输出格式等。当未指定时，将使用默认的搜索结果展示方式。最佳实践：1) 明确指定需要的输出格式（如列表、摘要等）；2) 指定结果排序方式（如按时间、相关性等）；3) 设置内容过滤条件（如排除特定类型的内容）。\n        temperature (float, optional): 控制模型输出随机性的采样参数，取值范围为(0, 1]。默认值为1e-10。该参数仅在model参数指定了LLM模型时生效。参数影响模型输出的多样性和稳定性：值越大，输出结果越多样化但可能不够稳定；值越小，输出结果越确定和集中。最佳实践：1) 对于事实性查询，建议使用较低的值（如0.1）；2) 对于创意性内容，可以使用较高的值（如0.7）；3) 避免使用极端值（接近0或1）。\n        top_p (float, opt

### 3.2 Request Model Example
In this section, the ERNIE 4.5 series models can be used directly for the following key parts:

- Deciding whether the main loop ends (if search further or not)
- Generating MCP search server calling parameters

You should firstly deploy a model as a local service in the following way and configure the model_url properly. 

#### 3.2.1 Deploying Model and Setting Key Parameters

The deployment can be accomplished using the FastDeploy tool, which is an inference deployment tool for large models open-sourced by PaddlePaddle. Currently, FastDeploy supports 4.5 series models, and provides commonly used inference deployment functionalities. For deployment methods, please refer to the [FastDeploy official documentation](https://github.com/PaddlePaddle/FastDeploy).

After deploying FastDeploy as a backend service, you need to set up the model_url in the configuration below.

In [42]:
model_url = "http://localhost:port/v1/chat/completions"

Prepare the following environment variables and parameters (as the default configuration for requesting model, which can be adjusted according to the actual situation of the used model):

In [43]:
api_key="bce-v3/ALTAK-xxx" # this could be the same as the API KEY you've got from QIANFAN in 3.1.1
model="ERNIE-4.5-xx" # Select the model to invoke
max_tokens=8000 # refer to the model documentation for the maximum number of tokens
temperature=0.7 # default setting or as per your requirement
top_p=1.0 # default setting or as per your requirement

The following is an example function for model request, please use it to test your model service.

In [44]:
from openai import OpenAI


def chat(query: str, model: str, max_tokens: int = 2048, temperature: float = 0.7, top_p: float = 1.0) -> str:
    """Process chat request and generate response.

    Args:
        query: User input query.
        model: Model name to use.
        max_tokens: Maximum tokens for response generation.
        temperature: Controls randomness of generated responses.
        top_p: Controls diversity of generated responses.

    Returns:
        str: response content.
    """
    conversation = []
    conversation.append({"role": "user", "content": query})
    try:
        client = OpenAI(base_url=model_url, api_key=api_key)
        response_text = client.chat.completions.create(
            model=model,
            messages=conversation,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p
        )
        response_text = response_text.model_dump()
        content_text = response_text["choices"][0]["message"]["content"]
        return content_text
    except Exception as e:
        raise print("Exception: " + repr(e))

## 4. Implement the Advanced Search Demo
This example implements a recursive AI-powered search loop demo designed to emulate expert reasoning and efficient information gathering.

### 4.1 Structure Overview
1. Input user query
2. Generate the task plan info
3. Iteratice Search (3-step loop)

        Step1: LLM Decision - if context information is enough
        
        Step2: Generates MCP Server Search Parameters
        
        Step3: Call MCP Server and Update Context

   
4. Answer user query with full context

Firstly, input your user query:

In [17]:
query = input("\nQuery: ").strip()


Query:  请帮我分析一下近年全球渐冻症领域的主要研究进展，包括发病机制、诊断手段、治疗方法等。请对比分析中美欧在诊疗技术上的差异，并梳理当前中国在渐冻症临床实践中面临的核心挑战。


### 4.2 Generate the Search Task Plan
Using LLM to generate a task plan based on the input query, to guide the subsequent search process.

In [45]:
import textwrap

# Generate task plan
task_plan = ""

TASK_PLAN_PROMPT = textwrap.dedent(
    """\
    # 任务
    我是一个专业的研究员。我会根据给定的query需求，逐层深入分析，并以清晰易懂的语言组织成一份完整的研究方案规划，便于执行和实施。
    ## 深度研究参考思路
    1. 目标确认与基础信息收集
        （1）确定研究目的：收集相关领域的信息，明确研究背景、细化通过研究想要达成的具体成果
        （2）数据收集与整理：我会系统收集目标领域的历史数据和案例，将这些信息整理成标准化的内容，可选择用数据表格形态整理。关键是确保数据的完整性、准确性和时序性，为后续所有分析提供可靠的事实基础，数据收集必须覆盖足够的时间范围\
    ，包含所有相关的关键信息字段。
    2. 深度分析与信息深度挖掘
        （1）深度模式分析：基于收集到的数据，我会深入分析其中的新研究对象、关键模式、规律和趋势等。这包括频率统计、周期性变化、发展趋势等量化分析，目标是揭示隐藏在数据背后的内在逻辑和规律性特征。对于上一步中出现的新的重要概念或\
    实体，需对该类需要探究的内容进行二次信息搜集。分析结果尽可能用统计数据和可视化图表来呈现。
        （2）核心驱动因素提取：通过对模式的深度分析，我需要识别出真正影响结果的核心驱动因素。这些因素需要按照影响力大小进行排序，并评估各自的权重。重点是找到那些具有决定性作用的关键变量，而不是表面的相关性因素。
        （3）现实背景信息补强：针对已识别的核心驱动因素，我会收集当前相关的现实背景信息。这包括最新的政策变化、市场环境、技术发展、社会趋势等可能影响分析结果的现实因素。目标是将历史规律与当前实际情况相结合，确保分析的时效性和准\
    确性。
    3. 研究成果产出
        （1）综合推理与结论：最后将整合上述获取到的所有信息，运用严密的逻辑推理得出最终结论。这不仅包括具体的预测结果，还要包含完整的推理过程、逻辑链条、以及对结论可信度的评估。同时需要考虑可能存在的风险因素和不确定性。
        （2）产出符合需求的交付形态：根据输入query需求或场景通用的形态，组织最终研究成果或调研报告，需包含上述研究过程的各类细节信息，
    *以上思路步骤仅供参考，可根据实际需求或命题情况进行调整


    ## query示例及样例输出：
    **输入query1：**
    "请你深入研究基于事件的光流估计，梳理经典代表算法与最新的研究趋势，分析当前这个领域的瓶颈及可行的解决方案"

    **样例输出1：**
    1. 资料收集
        (1) 调研基于事件的视觉传感器（事件相机）的工作原理、特点及其与传统帧式相机的区别。
        (2) 梳理基于事件的光流估计领域的经典代表算法，包括但不限于基于事件匹配、基于事件累积和基于学习的方法。
    2. 深入研究并梳理趋势并识别挑战
        (1) 深入分析基于事件的光流估计的最新研究趋势，例如与深度学习的结合、多模态融合以及在特定应用场景（如高速运动、低光照）下的进展。
        (2) 识别并分析当前基于事件的光流估计领域面临的主要瓶颈和挑战，例如数据稀疏性、噪声敏感性、计算效率、泛化能力以及缺乏大规模标准数据集。
    3. 瓶颈和方案探索
        (1) 针对上述瓶颈和挑战，探讨可行的解决方案和未来研究方向，例如新型事件表示方法、更鲁棒的算法设计、高效的硬件加速以及跨领域知识迁移。
        (2) 查找并分析基于事件的光流估计在实际应用中的案例，例如机器人导航、自动驾驶或工业检测，以了解其应用前景和实际需求。
    4. 报告撰写与交付
        （1）将研究成果整理成深入的研究报告，包括研究过程、结论和建议

    **输入query2：**
    "请帮我梳理一下面向AIGC/扩散模型/大模型训练的数据集所有权保护方法，并分析当前的挑战，给出一些创新性的思路方案"
    **样例输出2：**
    1. 信息收集和整理
        (1) 梳理当前适用于人工智能训练数据所有权和知识产权保护的法律和技术框架。
        (2) 深入研究专门针对AIGC、扩散模型和大型模型训练数据所有权保护的现有方法，例如数据水印、溯源技术、加密技术和合同协议。
    2. 分析和方案制定
        (1) 分析在大型AI模型训练中实施和执行这些数据所有权保护方法所面临的挑战，包括技术限制、法律模糊性及实际操作困难。
        (2) 查找并评估与AI模型训练数据所有权保护相关的实际案例或争议，以了解其成功和失败的经验。
        (3) 探索新兴技术和研究方向，如区块链、先进密码学或新型数字版权管理，它们如何为AI数据所有权保护提供创新解决方案。
        (4) 思考在AI数据所有权领域可能出现的政策或监管创新，例如新的许可模式、集体权利管理或国际合作框架。
    3. 报告撰写
        (1) 综合上述研究，提出结合法律、技术和伦理考量的创新性概念框架或混合方案，以实现AIGC/大型模型训练数据的稳健所有权保护。

    ## 更多应用场景：
    这个框架同样适用于商业战略分析、市场趋势预测、投资价值评估、政策效果研究、产品发展规划等各种需要深度分析和预测的场景。无论是分析企业发展策略、预测行业发展趋势，还是评估投资机会，都会按照同样的五层逻辑进行系统化的深度研究。

    ## 要求
    我不会对输入query作出任何评价，也不会解释相关的动作，我只会生成方案规划，一定不要直接回复输入query。
    我输出的规划是按步骤顺序组织的整体思路，每个步骤的规划用一两句话或一两个列表元素表示具体步骤，每个步骤的标题要严格和输入query相关，不要简单重复上述参考思路的小标题。
    但对于一些简单的问题，不用输出详细的规划步骤（也不需要给出规划示例）；如果遇到一些困难的问题，我可能会更进一步地扩展深度研究的要求。

    ## 当前输入query：{query}
    """
)

plan_prompt = TASK_PLAN_PROMPT.format(query="{query}")
task_plan = chat(plan_prompt.format(query=query), model, max_tokens, temperature, top_p)

print(task_plan)

1. 基础资料收集与整理
    (1) 发病机制研究进展：收集近年来全球关于渐冻症（ALS）发病机制的最新研究成果，涵盖基因突变（如SOD1、C9ORF72等）、蛋白质异常聚集（如TDP-43）、神经炎症、氧化应激及线粒体功能障碍等相关研究。
    (2) 诊断手段进展：整理目前全球范围内ALS的诊断标准及最新进展，包括神经电生理检测、影像学检查（如MRI）、生物标志物研究（如神经丝轻链蛋白）以及人工智能在诊断中的应用。
    (3) 治疗方法进展：调研近年来ALS治疗领域的重大突破，包括药物研发（如Relyvrio、Tofersen等）、基因治疗、干细胞治疗、呼吸支持及营养支持等非药物治疗手段的最新研究。
2. 中美欧诊疗技术差异对比分析
    (1) 诊断技术差异：对比中美欧在ALS诊断技术上的差异，包括诊断标准的执行情况、新型诊断技术的普及程度及诊断效率。
    (2) 治疗方法差异：分析中美欧在ALS治疗方法上的选择偏好及实施效果，特别是新药研发、临床试验的参与程度及非药物治疗手段的应用情况。
    (3) 医疗资源分配差异：探讨中美欧在ALS医疗资源分配上的差异，包括专科医生数量、治疗中心分布及患者获取治疗的便捷性。
3. 中国渐冻症临床实践核心挑战梳理
    (1) 诊断挑战：分析中国在ALS诊断过程中面临的主要挑战，包括诊断延误、误诊率、新型诊断技术的应用障碍等。
    (2) 治疗挑战：梳理中国在ALS治疗方面的核心挑战，包括新药可及性、治疗费用、非药物治疗手段的普及程度及患者治疗依从性。
    (3) 医疗资源与支持体系挑战：探讨中国在ALS医疗资源分配、患者支持体系（如护理、心理支持）及社会认知度方面存在的不足。
4. 深度分析与综合推理
    (1) 影响机制分析：基于收集的数据，深入分析ALS发病机制、诊断及治疗方法之间的相互影响，探讨不同诊疗技术对患者预后的影响。
    (2) 差异原因探究：针对中美欧在诊疗技术上的差异，探究其背后的原因，包括科研投入、医疗政策、社会文化等因素。
    (3) 挑战应对策略：结合中国渐冻症临床实践的核心挑战，探讨可能的应对策略，包括政策建议、技术创新、患者教育等。
5. 研究成果产出与报告撰写
    (1) 成果整合：将上述分析结果整合成一份全面的研究报告，涵盖ALS发病机制、诊断

### 4.3 LLM-Guided Iterative Search (3-Step Loop)
The overall process is then broken into three steps:

#### 4.3.1 Step 1: LLM Decision – Do We Have Enough Information?
First asks the LLM model to decicide:

>“Given the user's question and current context, is additional information needed before answering?”

This step ensures the demo can reason about knowledge gaps before querying any tool or database.

In [46]:
import json

# Step 1: Check if more information is needed
need_more_info = None
search_results=[]
SEARCH_INFO_PROMPT = textwrap.dedent(
    """\
    现在需要你完成以下 **[深度研究]任务**：
    **{query}**
    为了完成该 [深度研究] 任务，你需要通过**动态、多次地调用检索工具**，以获取足够且相关的 [参考信息](工具返回的搜索结果)。
    [参考信息] 必须满足以下两个要求：
    1. 全面覆盖任务问题中的各个细节需求；
    2. 对前序 [参考信息] 中涉及的延伸概念进行进一步的具体说明。

    在信息量不足或理解尚不充分的情况下，请持续调用工具补充信息，直到所收集的信息足以支撑对任务问题的完整理解和回答为止。

    你应当根据**历史检索结果**，判断下一步搜索的方向和重点，并决定是否需要继续调用工具。这一过程应具有**自适应性和递进性**。

    请严格参照以下任务规划指导，辅助你进行任务执行：
    ```
    {task_plan}
    ```
    历史检索结果：
    ```
    {search_results_string}
    ```

    请以JSON格式输出，格式要求如下：
    {{
        "reasoning": "决策理由",
        "need_more_tool_info": true/false
    }}
    注意：need_more_tool_info为true表示需要调用工具获取更多信息，若任务过于简单或需要向用户澄清获取信息时则为false。
    """
)
search_results_string = "\n\n".join(search_results)
content = chat(
    SEARCH_INFO_PROMPT.format(
        query=query, task_plan=task_plan, search_results_string=search_results_string
    ),
    model,
    max_tokens,
    temperature,
    top_p
)

try:
    start = content.find("{")
    end = content.rfind("}") + 1
    if start >= 0 and end > start:
        content = content[start:end]
    result = json.loads(content)
    print(f"Model decision:{result}")
except json.JSONDecodeError:
    print("错误：模型输出不是有效的JSON格式")

Model decision:{'reasoning': '为了完成深度研究任务，我将首先从基础资料收集与整理入手，具体包括发病机制、诊断手段和治疗方法的最新研究进展。这能帮助我建立对渐冻症（ALS）领域当前研究状况的全面理解。基于这些信息，我将对比分析中美欧在诊疗技术上的差异，包括诊断技术、治疗方法和医疗资源分配。接下来，我会根据这些对比分析，梳理中国在渐冻症临床实践中面临的核心挑战。最后，我会进行深度分析与综合推理，包括影响机制分析、差异原因探究和挑战应对策略的探讨，并将所有分析结果整合成一份全面的研究报告。因此，首先我需要调用工具，搜索全球在渐冻症发病机制、诊断手段、治疗方法方面的最新研究进展。', 'need_more_info': True}


#### 4.3.2 Step2: Generates MCP Server Search Parameters
If additional information is needed, the model is asked to generate precise search parameters for the MCP Server.

In [47]:
# Step 2: Generate search parameters
available_tools = json.dumps([{
    "name": tool.name,
    "description": tool.description,
    "input_schema": tool.inputSchema
} for tool in client.tools], ensure_ascii=False, indent=2)

search_results_string= "\n\n".join(search_results)

GEN_QUERY_PROMPT = textwrap.dedent(
    """\
    可用工具信息：
    {available_tools}

    历史检索结果：
    ```
    {search_results_string}
    ```
    用户问题：{query}

    请严格参照以下任务规划指导，执行下一步工具调用：
    ```
    {task_plan}
    ```

    请根据以上历史检索结果判断回复用户问题所缺少的信息内容，或根据任务规划指导判断下一步需要完成的检索任务，调用合适的工具，生成新的检索query来补充所缺少的信息或需要执行的其他检索任务.
    回复用户问题所需的完整参考信息需包含对用户问题中各细节需求及信息，以及参考信息中与需求相关的延伸概念的具体说明，同时需要完成任务规划中指导的各检索任务。
    生成检索query的要求如下：
    - 生成的检索query要与历史检索结果中的检索词不同，生成更深入的信息补充类query。
    - 单个检索的query需独立为一个简单直接的搜索主题，不能过长或过于复杂，若有多个需要补充的信息，则拆分成多个query，通过json列表形式分成多次工具调用。

    以下为生成本轮query的示例1：
    ```
    用户问题：帮我做竞品调研，深入了解当前Agent产品在旅行场景下如何识别、响应和解决用户的具体需求，包括需求识别的准确性、解决方案的有效性，以及在满足用户需求过程中面临的挑战。请结合当前市面上的产品和AI能力帮我输出详细的调研报告。
    上一轮检索query为："旅行场景 AI Agent 产品 竞品"
    上一轮得的检索结果包括以下内容："Manus"、"飞猪问一问"、"字节豆包AI"等Agent产品介绍
    本轮生成query则应根据上一轮检索结果进行下一步有关产品细节的深度挖掘，生成如下检索query：
    - Manus 需求应用案例
    - 飞猪问一问 需求应用案例
    - 字节豆包AI 需求应用案例
    ```

    以下为生成本轮query的示例2：
    ```
    用户问题：帮我做竞品调研，深入了解当前Agent产品在旅行场景下如何识别、响应和解决用户的具体需求，包括需求识别的准确性、解决方案的有效性，以及在满足用户需求过程中面临的挑战。请结合当前市面上的产品和AI能力帮我输出详细的调研报告。
    上一轮检索query为："Manus 需求方案 应用案例"、 "飞猪问一问 需求方案 应用案例"、"字节豆包AI 需求方案 应用案例"
    上一轮得的检索结果包括以下内容："Manus相关产品方案和案例"、"飞猪问一问应用案例"、"字节豆包AI产品方案"等产品案例内容
    本轮生成query则应根据上一轮检索结果判断，缺少有关产品满足用户需求中面临的挑战相关内容，因此进行下一步检索，生成如下检索query：
    - Manus 满足需求面临的挑战
    - 飞猪问一问 满足需求面临的挑战
    - 字节豆包AI 满足需求面临的挑战
    ```

    请根据用户问题和历史检索结果中的结果，参考任务规划指导进行工具调用，并参照上述示例生成恰当的参数，直接以JSON格式输出需要调用的工具名称（tool_name）和工具调用参数（tool_args）。
    """
)
content_text = chat(
    GEN_QUERY_PROMPT.format(
        available_tools=available_tools,
        query=query,
        task_plan=task_plan,
        search_results_string=search_results_string,
    ),
    model,
    max_tokens,
    temperature,
    top_p
)

try:
    start = content_text.find("```json") + 7
    end = content_text.rfind("```")
    if start >= 0 and end > start:
        content_text = content_text[start:end]
    content = json.loads(content_text)
    print(f"Model decision:{content}")
except json.JSONDecodeError:
    print("错误：模型输出不是有效的JSON格式")

Model decision:[{'tool_name': 'AIsearch', 'tool_args': {'query': '近年 全球渐冻症 发病机制 研究进展 基因突变 蛋白质异常聚集 神经炎症 氧化应激 线粒体功能障碍', 'instruction': '整理近年全球渐冻症（ALS）发病机制的研究进展，涵盖基因突变（如SOD1、C9ORF72等）、蛋白质异常聚集（如TDP-43）、神经炎症、氧化应激及线粒体功能障碍等相关研究，以列表形式展示'}}, {'tool_name': 'AIsearch', 'tool_args': {'query': '近年 全球渐冻症 诊断手段 研究进展 神经电生理检测 影像学检查 MRI 生物标志物研究 神经丝轻链蛋白 人工智能 诊断', 'instruction': '整理近年全球渐冻症（ALS）诊断手段的研究进展，包括神经电生理检测、影像学检查（如MRI）、生物标志物研究（如神经丝轻链蛋白）以及人工智能在诊断中的应用，以列表形式展示'}}, {'tool_name': 'AIsearch', 'tool_args': {'query': '近年 全球渐冻症 治疗方法 研究进展 药物研发 Relyvrio Tofersen 基因治疗 干细胞治疗 呼吸支持 营养支持', 'instruction': '整理近年全球渐冻症（ALS）治疗方法的研究进展，包括药物研发（如Relyvrio、Tofersen等）、基因治疗、干细胞治疗、呼吸支持及营养支持等非药物治疗手段的最新研究，以列表形式展示'}}, {'tool_name': 'AIsearch', 'tool_args': {'query': '中美欧 渐冻症 诊断技术 差异 诊断标准 新型诊断技术 普及程度 诊断效率', 'instruction': '对比中美欧在渐冻症（ALS）诊断技术上的差异，包括诊断标准的执行情况、新型诊断技术的普及程度及诊断效率，以列表形式展示'}}, {'tool_name': 'AIsearch', 'tool_args': {'query': '中美欧 渐冻症 治疗方法 差异 新药研发 临床试验 参与程度 非药物治疗手段 应用情况', 'instruction': '分析中美欧在渐冻症（ALS）治疗方法上的选择偏好及实施效果，特别是新药研发、

#### 4.3.3 Step3: Call MCP Server and Update Context
Using the parameters above, the system sends a request to the MCP Server and stores the results for reasoning in the next loop.

In [22]:
# Step 3: Call MCP server tool
import asyncio

import nest_asyncio


async def call_mcp(tool_name, tool_args):
    result = await client.call_tool(tool_name, tool_args)
    return result

nest_asyncio.apply() # 允许嵌套事件循环（只需运行一次）

loop = asyncio.get_event_loop()

search_result = []

for content_item in content:
    tool_name = content_item.get("tool_name")
    tool_args = content_item.get("tool_args")
    if "model" in tool_args:
        del tool_args["model"]

    tool_result = loop.run_until_complete(call_mcp(tool_name, tool_args))
    if any(text.type == "text" and "Error executing tool AIsearch" in text.text for text in tool_result.content):
        print(tool_result.content[0].text)
        break
    tool_result = "\n\n".join([text.text[:1000] for text in tool_result.content if text.type == "text"])
    print(f"工具调用结果(truncated):{tool_result}")
    search_result.append(f"检索词：{tool_args.get('query')}\n检索结果：\n{tool_result}\n")
search_results.append(f"***************第{len(search_results) + 1}次检索结果*******************：\n{json.dumps(search_result, ensure_ascii=False, indent=2)}\n")

工具调用结果(truncated):Title: 首款渐冻症对因药物开出首方!全球多款小核酸/基因/细胞疗法并进,ALS对因治疗迎加速突破
Content:  一、已上市药物: 从症状控制到病理干预的突破 目前,全球获批的 ALS 治疗药物仍以传统机制为主,但新型疗法已展现出对病理机制的针对性干预。 过去很长一段时间,ALS 的治疗目标以减缓病情进展和改善症状为主。但随着技术的不断发展,针对渐冻症的药物开发逐渐转为针对病因。 托夫生即是全球首个获批的渐冻症对因治疗药物。其通过靶向突变 SOD1 基因产生的 mRNA,可有效减少毒性 SOD1 蛋白的合成(这些毒性蛋白积累会引起细胞功能障碍,从而导致患者出现神经退化和肌肉无力)。 值得一提的是,6 月 10 日,托夫生已在北京大学第三医院开出首方,标志着这款药物正式应用于我国临床。 二、核心研发管线: 多技术平台并行,靶向精准治疗 当前ALS 研发管线呈现「多机制、多技术、全球化」特点,小核酸药物、基因疗法、细胞疗法等前沿技术与小分子、单抗等传统手段齐头并进。以下从作用机制角度梳理关键在研项目。 (一)小核酸药物 目前针对 ALS 开发的小核酸药物主要有ASO(反义寡核酸)
工具调用结果(truncated):Title: 渐冻症靶向治疗应用临床 患者在北医三院完成全国首针注射
Content:  2025年6月10日6:00,43岁的李女士在北京大学第三医院神经内科接受了托夫生注射液腰穿鞘内注射治疗。据了解,这是肌萎缩侧索硬化症(ALS,俗称“渐冻症”)靶向治疗药物托夫生注射液国内正式商业上市后的全国首方。 这标志着我国首个针对超氧化物歧化酶1(SOD1)基因突变所致成人渐冻症(SOD1-ALS)的疾病修正治疗药物正式应用于临床,为治疗这一罕见且致命的神经系统疾病带来源头干预的新希望。 据了解,李女士因不明原因的进行性肌肉无力来到北医三院就诊,诊断患有SOD1-ALS。在神经内科与药学部的紧密协作下,北医三院高效完成了药物引进和采购流程,确保了此次首方治疗的顺利实施。 托夫生注射液是全球首个且目前唯一获批用于SOD1-ALS成人患者的疾病修正治疗(DMT)药物。它突破了传统治疗仅能缓解症状的局限,首次实现了针对病因的精准干预。该药物分别于2023年4月获美国FDA批准、2024年9月获中国国家药品监督管理局(NMPA

Repeat the above three steps until the model's decision no longer requires additional information to be supplemented.

### 4.4 Generate Final Responses
At last, let LLM answer the user query with full context provived.

In [48]:
search_results_string= "\n\n".join(search_results)

FINAL_ANSWER_PROMPT = textwrap.dedent(
    """\
    参考信息：{reference_results_string}
    用户问题：{query}
    请参考以上信息回复用户问题，需遵循以下要求：
    1. 结合问题需求，对参考信息中的检索内容进行可用性判断，避免在回复中使用错误或不恰当的信息；
    2. 优先根据官网、百科、权威机构、专业网站等高权威性来源的信息来回答问题，更多地参考其中的相关数字和案例。
    3. 回复内容需覆盖用户需求的各个细节，尽可能的全面的解答问题，输出内容尽可能详细且结构化，按照用户需求的格式（如有）组织输出形式。
    """
)
final_response = chat(
    FINAL_ANSWER_PROMPT.format(
        reference_results_string=search_results_string,
        query=query
    ),
    model,
    max_tokens,
    temperature,
    top_p
)
print(f"final answer:{final_response}")

final answer:### 渐冻症领域近年（2020 年 - 至今）主要研究进展

#### 发病机制
- **基因研究深入**：近年来发现了多个与渐冻症（肌萎缩侧索硬化症，ALS）发病相关的新基因。例如，C9orf72 基因的六核苷酸重复扩增被认为是家族性 ALS 最常见的致病原因之一，这一发现有助于深入理解 ALS 的遗传基础和发病机制。此外，还有其他一些基因如 TARDBP、FUS 等也被证实与 ALS 发病相关，这些基因涉及 RNA 代谢、蛋白质稳态等重要生物学过程，其异常可能导致运动神经元的退化和死亡。
- **蛋白质聚集与毒性**：研究表明，异常蛋白质的聚集在 ALS 发病中起着关键作用。例如，TAR DNA 结合蛋白 43（TDP - 43）在大部分 ALS 患者的大脑和脊髓运动神经元中存在异常聚集，形成不溶性包涵体，干扰细胞的正常功能，最终导致神经元死亡。此外，超氧化物歧化酶 1（SOD1）突变引起的蛋白质错误折叠和聚集也被认为是 ALS 发病的重要机制之一。
- **神经炎症**：神经炎症在 ALS 发病过程中的作用受到越来越多的关注。小胶质细胞和星形胶质细胞的激活在 ALS 模型中被广泛观察到，这些激活的胶质细胞可以释放炎症因子，如肿瘤坏死因子 - α（TNF - α）、白细胞介素 - 6（IL - 6）等，进一步加重神经元的损伤和死亡。

#### 诊断手段
- **生物标志物研究**：寻找敏感和特异的生物标志物是 ALS 诊断研究的重要方向。近年来，一些潜在的生物标志物如神经丝轻链（NfL）在血液和脑脊液中的水平被证实与 ALS 的病情进展和预后相关。NfL 的检测有助于早期诊断和监测疾病的发展，为临床决策提供重要依据。
- **影像学技术**：先进的磁共振成像（MRI）技术，如扩散张量成像（DTI）和磁共振波谱成像（MRS），在 ALS 诊断中的应用越来越广泛。DTI 可以检测脑白质纤维束的完整性，发现 ALS 患者早期大脑运动区的结构改变；MRS 能够检测脑内代谢产物的变化，为 ALS 的诊断和鉴别诊断提供更多信息。

#### 治疗方法
- **药物治疗新进展**：虽然目前 ALS 仍然缺乏根治性的治疗方法，但近年来有一些新的药物进入临床试验阶段。例如，Relyvrio（苯丁酸钠和牛磺酸二醇口服固定剂量复方制剂）于 2022 年在

## 5. Iterative Search Implementation

Below is the example of the iterative search logic.

In [87]:
import json


async def AI_Deep_Search(query):

    # Generate task plan
    task_plan = ""
    TASK_PLAN_PROMPT = textwrap.dedent(
        """\
        # 任务
        我是一个专业的研究员。我会根据给定的query需求，逐层深入分析，并以清晰易懂的语言组织成一份完整的研究方案规划，便于执行和实施。
        ## 深度研究参考思路
        1. 目标确认与基础信息收集
            （1）确定研究目的：收集相关领域的信息，明确研究背景、细化通过研究想要达成的具体成果
            （2）数据收集与整理：我会系统收集目标领域的历史数据和案例，将这些信息整理成标准化的内容，可选择用数据表格形态整理。关键是确保数据的完整性、准确性和时序性，为后续所有分析提供可靠的事实基础，数据收集必须覆盖足够的时间范围\
        ，包含所有相关的关键信息字段。
        2. 深度分析与信息深度挖掘
            （1）深度模式分析：基于收集到的数据，我会深入分析其中的新研究对象、关键模式、规律和趋势等。这包括频率统计、周期性变化、发展趋势等量化分析，目标是揭示隐藏在数据背后的内在逻辑和规律性特征。对于上一步中出现的新的重要概念或\
        实体，需对该类需要探究的内容进行二次信息搜集。分析结果尽可能用统计数据和可视化图表来呈现。
            （2）核心驱动因素提取：通过对模式的深度分析，我需要识别出真正影响结果的核心驱动因素。这些因素需要按照影响力大小进行排序，并评估各自的权重。重点是找到那些具有决定性作用的关键变量，而不是表面的相关性因素。
            （3）现实背景信息补强：针对已识别的核心驱动因素，我会收集当前相关的现实背景信息。这包括最新的政策变化、市场环境、技术发展、社会趋势等可能影响分析结果的现实因素。目标是将历史规律与当前实际情况相结合，确保分析的时效性和准\
        确性。
        3. 研究成果产出
            （1）综合推理与结论：最后将整合上述获取到的所有信息，运用严密的逻辑推理得出最终结论。这不仅包括具体的预测结果，还要包含完整的推理过程、逻辑链条、以及对结论可信度的评估。同时需要考虑可能存在的风险因素和不确定性。
            （2）产出符合需求的交付形态：根据输入query需求或场景通用的形态，组织最终研究成果或调研报告，需包含上述研究过程的各类细节信息，
        *以上思路步骤仅供参考，可根据实际需求或命题情况进行调整


        ## query示例及样例输出：
        **输入query1：**
        "请你深入研究基于事件的光流估计，梳理经典代表算法与最新的研究趋势，分析当前这个领域的瓶颈及可行的解决方案"

        **样例输出1：**
        1. 资料收集
            (1) 调研基于事件的视觉传感器（事件相机）的工作原理、特点及其与传统帧式相机的区别。
            (2) 梳理基于事件的光流估计领域的经典代表算法，包括但不限于基于事件匹配、基于事件累积和基于学习的方法。
        2. 深入研究并梳理趋势并识别挑战
            (1) 深入分析基于事件的光流估计的最新研究趋势，例如与深度学习的结合、多模态融合以及在特定应用场景（如高速运动、低光照）下的进展。
            (2) 识别并分析当前基于事件的光流估计领域面临的主要瓶颈和挑战，例如数据稀疏性、噪声敏感性、计算效率、泛化能力以及缺乏大规模标准数据集。
        3. 瓶颈和方案探索
            (1) 针对上述瓶颈和挑战，探讨可行的解决方案和未来研究方向，例如新型事件表示方法、更鲁棒的算法设计、高效的硬件加速以及跨领域知识迁移。
            (2) 查找并分析基于事件的光流估计在实际应用中的案例，例如机器人导航、自动驾驶或工业检测，以了解其应用前景和实际需求。
        4. 报告撰写与交付
            （1）将研究成果整理成深入的研究报告，包括研究过程、结论和建议

        **输入query2：**
        "请帮我梳理一下面向AIGC/扩散模型/大模型训练的数据集所有权保护方法，并分析当前的挑战，给出一些创新性的思路方案"
        **样例输出2：**
        1. 信息收集和整理
            (1) 梳理当前适用于人工智能训练数据所有权和知识产权保护的法律和技术框架。
            (2) 深入研究专门针对AIGC、扩散模型和大型模型训练数据所有权保护的现有方法，例如数据水印、溯源技术、加密技术和合同协议。
        2. 分析和方案制定
            (1) 分析在大型AI模型训练中实施和执行这些数据所有权保护方法所面临的挑战，包括技术限制、法律模糊性及实际操作困难。
            (2) 查找并评估与AI模型训练数据所有权保护相关的实际案例或争议，以了解其成功和失败的经验。
            (3) 探索新兴技术和研究方向，如区块链、先进密码学或新型数字版权管理，它们如何为AI数据所有权保护提供创新解决方案。
            (4) 思考在AI数据所有权领域可能出现的政策或监管创新，例如新的许可模式、集体权利管理或国际合作框架。
        3. 报告撰写
            (1) 综合上述研究，提出结合法律、技术和伦理考量的创新性概念框架或混合方案，以实现AIGC/大型模型训练数据的稳健所有权保护。

        ## 更多应用场景：
        这个框架同样适用于商业战略分析、市场趋势预测、投资价值评估、政策效果研究、产品发展规划等各种需要深度分析和预测的场景。无论是分析企业发展策略、预测行业发展趋势，还是评估投资机会，都会按照同样的五层逻辑进行系统化的深度研究。

        ## 要求
        我不会对输入query作出任何评价，也不会解释相关的动作，我只会生成方案规划，一定不要直接回复输入query。
        我输出的规划是按步骤顺序组织的整体思路，每个步骤的规划用一两句话或一两个列表元素表示具体步骤，每个步骤的标题要严格和输入query相关，不要简单重复上述参考思路的小标题。
        但对于一些简单的问题，不用输出详细的规划步骤（也不需要给出规划示例）；如果遇到一些困难的问题，我可能会更进一步地扩展深度研究的要求。

        ## 当前输入query：{query}
        """
    )

    plan_prompt = TASK_PLAN_PROMPT.format(query="{query}")
    task_plan = chat(plan_prompt, model, max_tokens, temperature, top_p)
    print(f"task plan:{task_plan}")

    # Start iterative Search

    current_iteration = 0
    max_iterations = 5
    search_results=[]

    while current_iteration < max_iterations:
        current_iteration += 1
        # Step 1: Check if more information is needed
        need_more_info = None

        SEARCH_INFO_PROMPT = textwrap.dedent(
            """\
            现在需要你完成以下 **[深度研究]任务**：
            **{query}**
            为了完成该 [深度研究] 任务，你需要通过**动态、多次地调用检索工具**，以获取足够且相关的 [参考信息](工具返回的搜索结果)。
            [参考信息] 必须满足以下两个要求：
            1. 全面覆盖任务问题中的各个细节需求；
            2. 对前序 [参考信息] 中涉及的延伸概念进行进一步的具体说明。

            在信息量不足或理解尚不充分的情况下，请持续调用工具补充信息，直到所收集的信息足以支撑对任务问题的完整理解和回答为止。

            你应当根据**历史检索结果**，判断下一步搜索的方向和重点，并决定是否需要继续调用工具。这一过程应具有**自适应性和递进性**。

            请严格参照以下任务规划指导，辅助你进行任务执行：
            ```
            {task_plan}
            ```
            历史检索结果：
            ```
            {search_results_string}
            ```

            请以JSON格式输出，格式要求如下：
            {{
                "reasoning": "决策理由",
                "need_more_tool_info": true/false
            }}
            注意：need_more_tool_info为true表示需要调用工具获取更多信息，若任务过于简单或需要向用户澄清获取信息时则为false。
            """
        )
        search_results_string= "\n\n".join(search_results)
        content = chat(
            SEARCH_INFO_PROMPT.format(
                query=query, task_plan=task_plan, search_results_string=search_results_string
            ),
            model,
            max_tokens,
            temperature,
            top_p
        )

        try:
            start = content.find("{")
            end = content.rfind("}") + 1
            if start >= 0 and end > start:
                content = content[start:end]
            result = json.loads(content)
            print(f"Model decision:{result}")
        except json.JSONDecodeError:
            print("错误：模型输出不是有效的JSON格式")
            continue

        if not result:
            continue

        if not result.get("need_more_tool_info", False):
            print("信息搜集完毕，终止检索")
            break

        print(f"************开始第{current_iteration}次检索************")
        # Step 2: Generate search parameters
        available_tools = json.dumps([{
            "name": tool.name,
            "description": tool.description,
            "input_schema": tool.inputSchema
        } for tool in client.tools], ensure_ascii=False, indent=2)

        GEN_QUERY_PROMPT = textwrap.dedent(
            """\
            可用工具信息：
            {available_tools}

            历史检索结果：
            ```
            {search_results_string}
            ```
            用户问题：{query}

            请严格参照以下任务规划指导，执行下一步工具调用：
            ```
            {task_plan}
            ```

            请根据以上历史检索结果判断回复用户问题所缺少的信息内容，或根据任务规划指导判断下一步需要完成的检索任务，调用合适的工具，生成新的检索query来补充所缺少的信息或需要执行的其他检索任务.
            回复用户问题所需的完整参考信息需包含对用户问题中各细节需求及信息，以及参考信息中与需求相关的延伸概念的具体说明，同时需要完成任务规划中指导的各检索任务。
            生成检索query的要求如下：
            - 生成的检索query要与历史检索结果中的检索词不同，生成更深入的信息补充类query。
            - 单个检索的query需独立为一个简单直接的搜索主题，不能过长或过于复杂，若有多个需要补充的信息，则拆分成多个query，通过json列表形式分成多次工具调用。

            以下为生成本轮query的示例1：
            ```
            用户问题：帮我做竞品调研，深入了解当前Agent产品在旅行场景下如何识别、响应和解决用户的具体需求，包括需求识别的准确性、解决方案的有效性，以及在满足用户需求过程中面临的挑战。请结合当前市面上的产品和AI能力帮我输出详细的调研报告。
            上一轮检索query为："旅行场景 AI Agent 产品 竞品"
            上一轮得的检索结果包括以下内容："Manus"、"飞猪问一问"、"字节豆包AI"等Agent产品介绍
            本轮生成query则应根据上一轮检索结果进行下一步有关产品细节的深度挖掘，生成如下检索query：
            - Manus 需求应用案例
            - 飞猪问一问 需求应用案例
            - 字节豆包AI 需求应用案例
            ```

            以下为生成本轮query的示例2：
            ```
            用户问题：帮我做竞品调研，深入了解当前Agent产品在旅行场景下如何识别、响应和解决用户的具体需求，包括需求识别的准确性、解决方案的有效性，以及在满足用户需求过程中面临的挑战。请结合当前市面上的产品和AI能力帮我输出详细的调研报告。
            上一轮检索query为："Manus 需求方案 应用案例"、 "飞猪问一问 需求方案 应用案例"、"字节豆包AI 需求方案 应用案例"
            上一轮得的检索结果包括以下内容："Manus相关产品方案和案例"、"飞猪问一问应用案例"、"字节豆包AI产品方案"等产品案例内容
            本轮生成query则应根据上一轮检索结果判断，缺少有关产品满足用户需求中面临的挑战相关内容，因此进行下一步检索，生成如下检索query：
            - Manus 满足需求面临的挑战
            - 飞猪问一问 满足需求面临的挑战
            - 字节豆包AI 满足需求面临的挑战
            ```

            请根据用户问题和历史检索结果中的结果，参考任务规划指导进行工具调用，并参照上述示例生成恰当的参数，直接以JSON格式输出需要调用的工具名称（tool_name）和工具调用参数（tool_args）。
            """
        )
        content_text = chat(
            GEN_QUERY_PROMPT.format(
                available_tools=available_tools,
                query=query,
                task_plan=task_plan,
                search_results_string=search_results_string,
            ),
            model,
            max_tokens,
            temperature,
            top_p
        )

        try:
            start = content_text.find("```json") + 7
            end = content_text.rfind("```")
            if start >= 0 and end > start:
                content_text = content_text[start:end]
            content = json.loads(content_text)
            print(f"Model decision:{content}")
        except json.JSONDecodeError:
            print("错误：模型输出不是有效的JSON格式")
            continue

        if not content:
            continue

        # Step 3: Call MCP server tool
        search_result = []

        for content_item in content:
            tool_name = content_item.get("tool_name")
            tool_args = content_item.get("tool_args")
            if "model" in tool_args:
                del tool_args["model"]
            for key in list(tool_args.keys()):  # 复制键，生成列表
                if tool_args[key] == "" or tool_args[key] is None:
                    del tool_args[key]
            print(tool_args)
            tool_result = await client.call_tool(tool_name, tool_args)
            tool_result = "\n\n".join([text.text[:2000] for text in tool_result.content if text.type == "text"])
            print(f"工具调用结果(truncated):{tool_result}")
            search_result.append(f"检索词：{tool_args.get('query')}\n检索结果：\n{tool_result}\n")
            search_result_string = "\n\n".join(search_result)
        search_results.append(f"***************第{len(search_results) + 1}次检索结果*******************：\n{search_result_string}\n")

        print(f"************第{current_iteration}次检索完成************")

    print("************输出最终结果***********")

    search_results_string= "\n\n".join(search_results)

    FINAL_ANSWER_PROMPT = textwrap.dedent(
        """\
        参考信息：{reference_results_string}
        用户问题：{query}
        请参考以上信息回复用户问题，需遵循以下要求：
        1. 结合问题需求，对参考信息中的检索内容进行可用性判断，避免在回复中使用错误或不恰当的信息；
        2. 优先根据官网、百科、权威机构、专业网站等高权威性来源的信息来回答问题，更多地参考其中的相关数字和案例。
        3. 回复内容需覆盖用户需求的各个细节，尽可能的全面的解答问题，输出内容尽可能详细且结构化，按照用户需求的格式（如有）组织输出形式。
        """
    )
    final_response = chat(
        FINAL_ANSWER_PROMPT.format(
            reference_results_string=search_results_string,
            query=query
        ),
        model,
        max_tokens,
        temperature,
        top_p
    )
    print(f"final answer:{final_response}")
    return final_response

#### Example case

In [90]:
nest_asyncio.apply() # 允许嵌套事件循环（只需运行一次）
loop = asyncio.get_event_loop()
result = loop.run_until_complete(AI_Deep_Search("请对基于大语言模型（LLM）的文本摘要事实一致性评估方法展开系统调研，重点梳理近年来的最新研究进展与代表性工作，全面分析当前在无监督事实一致性评估中的技术路径、能力边界与主要挑战。在此基础上，结合现有技术瓶颈，提出两至三个具有较高可行性的无监督事实一致性评估创新思路，并通过框架图形式清晰展示每个思路的整体流程与关键模块设计"))

task plan:1. 文献与现状调研
    (1) 系统收集近年来基于大语言模型（LLM）的文本摘要事实一致性评估方法的相关文献，涵盖学术会议论文、期刊论文、技术报告等。
    (2) 整理并分析这些文献中的最新研究进展，提炼出具有代表性的研究工作，明确不同方法的核心思想、技术特点及优缺点。
2. 无监督事实一致性评估技术路径、能力边界与挑战分析
    (1) 对无监督事实一致性评估中的技术路径进行详细梳理，包括但不限于基于语义相似度、基于知识图谱、基于逻辑推理等方法。
    (2) 评估不同技术路径在处理文本摘要事实一致性评估时的能力边界，明确各种方法在不同场景下的适用性。
    (3) 全面分析当前无监督事实一致性评估面临的主要挑战，如语义理解的深度不足、知识覆盖的局限性、评估指标的合理性等。
3. 创新思路提出
    (1) 结合现有技术瓶颈，提出两至三个具有较高可行性的无监督事实一致性评估创新思路，如融合多模态信息的评估方法、基于对抗训练的评估模型等。
    (2) 针对每个创新思路，详细阐述其设计理念、技术实现路径及预期效果。
4. 创新思路框架图设计
    (1) 为每个创新思路设计框架图，清晰展示整体流程与关键模块设计，包括输入、处理过程、输出等环节。
    (2) 在框架图中明确标注各模块的功能及相互之间的关联，确保框架图的可读性和可理解性。
5. 案例验证与效果评估（可选，根据实际需求与资源决定是否实施）
    (1) 选取具有代表性的文本摘要数据集，对提出的创新思路进行实证研究，验证其有效性和可行性。
    (2) 设计合理的评估指标，对创新思路在无监督事实一致性评估中的效果进行量化评估，对比不同方法之间的性能差异。
6. 报告撰写与成果总结
    (1) 将上述调研、分析、创新思路及框架图等内容整理成系统调研报告，确保报告内容完整、逻辑清晰、数据准确。
    (2) 总结研究成果，明确未来研究方向，提出对基于大语言模型的文本摘要事实一致性评估方法的改进建议。
Model decision:{'need_more_info': True, 'reasoning': '为了完成基于大语言模型（LLM）的文本摘要事实一致性评估方法的系统调研，首先需要系统收集近年来相关的学术会议论文、期刊论文、技术报告等文献，以全面覆盖任

## 6. Summary
This tutorial demonstrates how to build a model-guided, recursive AI search demo by:

- Using an LLM to reason and guide the search
- Dynamically generating search parameters
- Calling the MCP Server to retrieve relevant knowledge
- Closing the loop when sufficient information is gathered

This pattern is extensible and production-ready for intelligent enterprise search workflows.